In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('outbreaks.csv')

In [4]:
df = df[df['Food'].notnull()][df['Species'].notnull()]

/home/land/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [6]:
df.head()

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
3,1998,January,California,Restaurant,"Fish, Ahi",NaN,Scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"Lasagna, Unspecified; Eggs, Other",NaN,Salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0
7,1998,January,California,Restaurant,"Stuffing, Unspecified; Sandwich, Turkey",NaN,Salmonella enterica,Enteritidis,Confirmed,4,3.0,0.0
8,1998,January,Colorado,Restaurant,"Salad, Unspecified",NaN,Virus,NaN,Suspected,21,NaN,NaN
9,1998,January,Colorado,Restaurant,"Peppers, Chili",NaN,Virus,NaN,Suspected,3,NaN,0.0


In [52]:
food_set = df['Food'].str.lower().str.replace(',','').str.replace(';','').str.split()
food_set.head()

3                                  [fish, ahi]
4          [lasagna, unspecified, eggs, other]
7    [stuffing, unspecified, sandwich, turkey]
8                         [salad, unspecified]
9                             [peppers, chili]
Name: Food, dtype: object

In [53]:
species_set = df['Species'].str.lower().str.replace(';',',').str.split(',').map(lambda x:[i.strip() for i in x])
species_set.head()

3        [scombroid toxin]
4    [salmonella enterica]
7    [salmonella enterica]
8                  [virus]
9                  [virus]
Name: Species, dtype: object

In [61]:
all_foods = set()
for i in food_set:
    all_foods |= set(i)
len(all_foods)

1041

In [62]:
all_toxins = set()
for i in species_set:
    all_toxins |= set(i)
len(all_toxins)

80

In [63]:
food_toxin_matrix = pd.DataFrame(columns = all_foods, index = all_toxins, data = 0)
food_toxin_matrix.head()

,octopus,hot,vegetable,patties,pasteurized,meatloaf,stuffings,yellowtail,turshi,mamey,...,bruschetta,curds,sea,gazpacho,susumber,quiche,skipjack,alfredo,shumai,gelatin
bacillus other,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
giardia intestinalis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
histamine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adenovirus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
trichinella unknown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
for food, toxin in zip(food_set, species_set):
    for f in food:
        for t in toxin:
            food_toxin_matrix.loc[t,f] +=1

In [68]:
food_toxin_matrix.sum(0), food_toxin_matrix.sum(1)

(octopus         3
 hot            29
 vegetable      71
 patties         2
 pasteurized    48
                ..
 quiche          4
 skipjack        1
 alfredo         9
 shumai          1
 gelatin         6
 Length: 1041, dtype: int64, bacillus other                   11
 giardia intestinalis             17
 histamine                       102
 adenovirus                        2
 trichinella unknown               5
                                ... 
 neurotoxic shellfish poison      16
 giardia                           4
 chemical or toxin               247
 norovirus genogroup i          3684
 clostridium botulinum           129
 Length: 80, dtype: int64)